In [1]:
# !pip install transformers
# !pip install catboost

# from google.colab import drive
# drive.mount("/content/drive")
# data

In [4]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [5]:
import time
time.sleep(2)

In [6]:
data_path='/datasets/'
df = pd.read_csv(f"{data_path}toxic_comments.csv",index_col=0,encoding='utf-8')
df = df.sample(5000).reset_index()
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   5000 non-null   int64 
 1   text    5000 non-null   object
 2   toxic   5000 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 117.3+ KB


,index,text,toxic
0,12348,"""\n\nArticle rating\n\nI am rating this articl...",0
1,29981,Nobody said that. But you have a COI and you a...,0
2,5712,642003399&oldid;=642001720 He attacked me at a...,0
3,57105,Of course we cannot cut and paste the enitre b...,0
4,95757,Baichung Bhutia \n\nI've replied to you on the...,0


In [7]:
model_class, tokenizer_class, pretrained_weights = (ppb.MobileBertModel, ppb.MobileBertTokenizer, 'google/mobilebert-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights,)
model = model_class.from_pretrained(pretrained_weights)

In [8]:
%%time
tokenized = df.text.apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
tokenized

CPU times: user 3.79 s, sys: 21.5 ms, total: 3.81 s
Wall time: 3.39 s


0       [101, 1000, 3720, 5790, 1045, 2572, 5790, 2023...
1       [101, 6343, 2056, 2008, 1012, 2021, 2017, 2031...
2       [101, 4185, 28332, 22394, 2683, 2683, 1004, 22...
3       [101, 1997, 2607, 2057, 3685, 3013, 1998, 1935...
4       [101, 21790, 20760, 3070, 1038, 6979, 10711, 1...
                              ...                        
4995    [101, 11721, 2128, 27241, 4757, 3672, 1997, 13...
4996    [101, 3749, 1997, 2393, 4283, 999, 1045, 2428,...
4997    [101, 1000, 2061, 1045, 2001, 2157, 999, 1000,...
4998    [101, 9805, 2361, 1010, 2002, 1005, 1055, 1519...
4999    [101, 1000, 15501, 6633, 1010, 1000, 1000, 614...
Name: text, Length: 5000, dtype: object

In [9]:
%%time
#max_len = int( np.quantile( list( map(len,tokenized)),.99) )
max_len = 512
padded = np.array([ np.array(row[:max_len] + [0]*(max_len - len(row)),dtype=np.float16) for row in tokenized ])
padded

CPU times: user 99.1 ms, sys: 36 µs, total: 99.1 ms
Wall time: 98.2 ms


array([[  101.,  1000.,  3720., ...,     0.,     0.,     0.],
       [  101.,  6344.,  2056., ...,     0.,     0.,     0.],
       [  101.,  4184., 28336., ...,     0.,     0.,     0.],
       ...,
       [  101.,  1000.,  2060., ...,     0.,     0.,     0.],
       [  101.,  9808.,  2360., ...,     0.,     0.,     0.],
       [  101.,  1000., 15504., ...,     0.,     0.,     0.]],
      dtype=float16)

In [10]:
attention_mask = np.where(padded != 0, 1, 0)

In [ ]:
from tqdm import notebook
batch_size = 25
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        time.sleep(5)
        !cat /proc/meminfo | grep Mem
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
        attention_mask_batch = torch.Tensor(attention_mask[batch_size*i:batch_size*(i+1)])

        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)

        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/200 [00:00<?, ?it/s]

MemTotal:        3856324 kB
MemFree:         1962760 kB
MemAvailable:    2588552 kB
MemTotal:        3856324 kB
MemFree:         1293444 kB
MemAvailable:    1925892 kB
MemTotal:        3856324 kB
MemFree:         1202552 kB
MemAvailable:    1835000 kB
MemTotal:        3856324 kB
MemFree:         1067556 kB
MemAvailable:    1700004 kB
MemTotal:        3856324 kB
MemFree:         1067068 kB
MemAvailable:    1699516 kB
MemTotal:        3856324 kB
MemFree:          967588 kB
MemAvailable:    1600036 kB
MemTotal:        3856324 kB
MemFree:          963092 kB
MemAvailable:    1595696 kB
MemTotal:        3856324 kB
MemFree:          958484 kB
MemAvailable:    1591096 kB
MemTotal:        3856324 kB
MemFree:          958812 kB
MemAvailable:    1591432 kB
MemTotal:        3856324 kB
MemFree:          728924 kB
MemAvailable:    1361544 kB
MemTotal:        3856324 kB
MemFree:          570120 kB
MemAvailable:    1202740 kB
MemTotal:        3856324 kB
MemFree:          544980 kB
MemAvailable:    117

In [ ]:
features = np.concatenate(embeddings)

In [ ]:
X_tr,X_te,y_tr,y_te = train_test_split(features,df.toxic,test_size=.33)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,confusion_matrix
model = LogisticRegression()
model.fit(X_tr,y_tr)
pr = model.predict(X_te)

In [ ]:
confusion_matrix(y_te,pr)

In [ ]:
features.shape

In [ ]:
df.toxic.sum()

In [ ]:
prb = model.predict_proba(X_te)

In [ ]:
from catboost import CatBoostClassifier